In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import pickle

In [2]:
df = pd.read_csv('../data/interim/train_clean_roll_up.csv')

In [3]:
df.head()

,ProductName,RtpStateBitfield,IsSxsPassiveMode,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,CityIdentifier,OrganizationIdentifier,...,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections,Census_OSBranch_release_extracted,EngineVersion_encoded,AppVersion_encoded,AvSigVersion_encoded,OsVer_encoded,Census_OSVersion_encoded
0,win8defender,7.0,0,53447.0,1.0,1.0,1,94,65823.0,NaN,...,0.0,0.0,11.0,0,rs4_release,15200,18,275,1,17134
1,win8defender,7.0,0,23657.0,2.0,1.0,1,139,87570.0,NaN,...,0.0,1.0,1.0,0,rs4_release,15100,18,273,1,17134
2,win8defender,7.0,0,23962.0,2.0,1.0,1,123,94381.0,NaN,...,0.0,0.0,5.0,0,rs4_release,15200,18,275,1,17134
3,win8defender,7.0,0,46669.0,2.0,1.0,1,85,NaN,27.0,...,0.0,0.0,10.0,1,th,15100,Other,273,1,10586
4,win8defender,7.0,0,53447.0,1.0,1.0,1,50,105713.0,27.0,...,0.0,0.0,3.0,1,rs4_release,15100,18,273,1,17134


In [4]:
df.shape

(89212, 66)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89212 entries, 0 to 89211
Data columns (total 66 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   ProductName                                        89211 non-null  object 
 1   RtpStateBitfield                                   88876 non-null  float64
 2   IsSxsPassiveMode                                   89212 non-null  int64  
 3   AVProductStatesIdentifier                          88847 non-null  float64
 4   AVProductsInstalled                                88847 non-null  float64
 5   AVProductsEnabled                                  88847 non-null  float64
 6   HasTpm                                             89212 non-null  int64  
 7   CountryIdentifier                                  89212 non-null  int64  
 8   CityIdentifier                                     85949 non-null  float64
 9   Organi

In [6]:
col_nan = df.isnull().mean()
col_nan.sort_values(ascending=False)[:10]

SmartScreen                              0.358730
OrganizationIdentifier                   0.306360
CityIdentifier                           0.036576
Wdft_RegionIdentifier                    0.032944
Wdft_IsGamer                             0.032944
Census_InternalBatteryNumberOfCharges    0.030332
Census_FirmwareManufacturerIdentifier    0.021286
Census_FirmwareVersionIdentifier         0.018596
Census_OEMModelIdentifier                0.012263
Census_OEMNameIdentifier                 0.011366
dtype: float64

In [7]:
categorical_columns = df.select_dtypes('object').columns.tolist()
binary_columns = [col for col in df.columns.tolist() 
                  if df[col].nunique() <= 2 
                  if df[col].dtypes != 'O']
true_numerical_columns = ['Census_ProcessorCoreCount',
                          'Census_PrimaryDiskTotalCapacity',
                          'Census_SystemVolumeTotalCapacity',
                          'Census_TotalPhysicalRAM',
                          'Census_InternalPrimaryDiagonalDisplaySizeInInches',
                          'Census_InternalPrimaryDisplayResolutionHorizontal',
                          'Census_InternalPrimaryDisplayResolutionVertical',
                          'Census_InternalBatteryNumberOfCharges']
numerical_columns = true_numerical_columns
cat_num_columns = [col for col in df.columns.tolist() 
                   if col not in numerical_columns 
                   if col not in binary_columns 
                   if col not in categorical_columns]
tot_columns = numerical_columns + binary_columns + categorical_columns + cat_num_columns
len(categorical_columns), len(binary_columns), len(numerical_columns), len(cat_num_columns), len(tot_columns)

(24, 11, 8, 23, 66)

In [8]:
col_n_uniq = df[categorical_columns].nunique()
col_n_uniq.sort_values(ascending=False)

Census_MDC2FormFactor                11
OsPlatformSubRelease                  9
Census_PowerPlatformRoleName          9
Census_OSInstallTypeName              9
SkuEdition                            8
SmartScreen                           8
Census_OSVersion_encoded              7
Census_FlightRing                     7
Census_OSBranch_release_extracted     6
Census_ActivationChannel              6
Census_OSWUAutoUpdateOptionsName      6
Census_OSSkuName                      4
Census_GenuineStateName               4
Census_OSEdition                      4
Platform                              4
Census_ChassisTypeName                4
Census_PrimaryDiskTypeName            3
EngineVersion_encoded                 3
AvSigVersion_encoded                  3
Census_OSArchitecture                 2
Census_DeviceFamily                   2
Processor                             2
AppVersion_encoded                    2
ProductName                           2
dtype: int64

In [9]:
'HasDetections' in tot_columns

True

In [10]:
target_index = tot_columns.index('HasDetections')
target_name = tot_columns.pop(target_index)

In [11]:
'HasDetections' in tot_columns

False

In [12]:
'HasDetections' in binary_columns

True

In [13]:
idx = binary_columns.index('HasDetections')
binary_columns.pop(idx)

'HasDetections'

In [14]:
'HasDetections' in binary_columns

False

In [15]:
X = df[tot_columns]
y = df[target_name]

In [16]:
#Seperate train and test data
X_train, X_test, y_train, y_test = train_test_split(X,
                                                   y,
                                                   test_size = 0.2,
                                                   random_state = 10)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(71369, 65)
(17843, 65)
(71369,)
(17843,)


In [17]:
y_train.mean(), y_test.mean()

(0.5001751460718239, 0.5040632180687105)

In [18]:
with open('../data/processed/X_train', 'wb') as handle:
    pickle.dump(X_train,handle)
with open('../data/processed/X_test', 'wb') as handle:
    pickle.dump(X_test, handle)
with open('../data/processed/y_train', 'wb') as handle:
    pickle.dump(y_train, handle)
with open('../data/processed/y_test', 'wb') as handle:
    pickle.dump(y_test, handle)